# CBAR Miniproject : code

In [ ]:
# All import for the project
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np 
import random
import heapq
import statsmodels as sm
import matplotlib as mpl
import matplotlib.cm as cm

from scipy.ndimage import gaussian_filter1d
from scipy.stats import gaussian_kde
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from behavelet import wavelet_transform
import scipy.stats as st
from scipy.stats import zscore
from itertools import combinations
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression


np.set_printoptions(precision=2, suppress=True)

%matplotlib inline

random.seed(36)

In [ ]:
#Let's first load the neural data with pandas and look at it
neural_data = pd.read_pickle('data/COBAR_neural.pkl')
neural_data

In [ ]:
# Let's plot traces from individual neurons 
# We will use random number generation to choose few neurons to plot as we don't want to plot all the 123 neurons
# We've chosen here to plot 5 neurons

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]
plot_scaling_factor = 5000

for j in range(5):
    random_neuron = random.randrange(122)
    neuron = "neuron_"+str(random_neuron)

    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data[neural_data.index.get_level_values("Trial")==0]["t"], i*plot_scaling_factor + neural_data[neuron][neural_data.index.get_level_values("Trial")==i].values - neural_data[neuron][neural_data.index.get_level_values("Trial")==i].values[0])
        ytick.append(i*plot_scaling_factor)

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Fluorescence [-]")
    ax.set_title(f'Fluorescence data of neuron {random_neuron} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
neural_data.describe()

In [ ]:
# As we see here and in the traces, a good summary of a neuron would be his standard deviation as it does 
# show whether the neuron is active during the trials.
#Let's find the neurons with the maximal standard deviation
maximums = []
maximum_values = []
for neuron in neural_data : 
    if not neuron == 't' : 
        maximums.append(neural_data.groupby('Trial').std().nlargest(1, neuron).index.values[0])
        maximum_values.append(neural_data.groupby('Trial').std().nlargest(1, neuron)[neuron].values[0])

In [ ]:
# And plot them
ten_max_values = heapq.nlargest(10, maximum_values)
maximum_neuron = []
maximum_trial = []
for value in ten_max_values :
    maximum_neuron.append(maximum_values.index(value))
    maximum_trial.append(maximums[maximum_values.index(value)])


ytick = []

fig, ax = plt.subplots(figsize = (16,6))

i=0
for index in maximum_neuron:
    neuron = neural_data[neural_data.columns[index+1]].name
    temp = np.copy(neural_data[neural_data.index.get_level_values('Trial')==maximums[index]][neuron])
    ax.plot(neural_data[neural_data.index.get_level_values('Trial')==maximums[index]]['t'], i*5000 + temp - temp[0], label = neuron)
    ytick.append(i*5000)
    i=i+1

ax.set_xlabel("Time [s]")
ax.set_ylabel("Fluorescence [-]")
ax.set_title(f'Fluorescence data of the ten most active neurons across trials')

index_name = ['neuron_' + str(maximum_neuron[i]) +'_trial_'+ str(maximum_trial[i]) for i in range(len(maximum_neuron))]
plt.yticks(np.array(ytick), index_name)
plt.grid()
plt.show()

In [ ]:
#Let's now load the behavioral data with pandas and look at it
behav_data = pd.read_pickle('data/COBAR_behaviour_incl_manual_corrected.pkl')
behav_data

In [ ]:
# We can see the differences between 
print(len(behav_data['Prediction'].unique()), len(behav_data['Manual'].unique()))

In [ ]:
# We observe that joints and angles are represented in the columns 2 to 134.
behav_data.columns[2:134]

In [ ]:
# Let's plot traces from individual joints or angles 
# We've chosen here to plot 5 angles or joints randomly

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]

for j in range(5) :
    random_mvmt = random.randrange(133)
    mvmt = behav_data.columns[random_mvmt+2]

    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(behav_data[behav_data.index.get_level_values("Trial")==0]["t"], i*plot_scaling_factor + (behav_data[mvmt][behav_data.index.get_level_values("Trial")==i].values - behav_data[mvmt][behav_data.index.get_level_values("Trial")==i].values[0])*plot_scaling_factor/2.1)
        ytick.append(i*plot_scaling_factor)

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Joint angle [rad] / position [mm]")
    ax.set_title(f'Joint angle or position of {mvmt} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
# The minimum of the fluorescence would not be a good baseline thus we will compute a moving average 
# with help of the pandas Series function rolling with a window size of 10. 

# Here we computed the baseline for each neuron for each trial and store it in a new dataframe 

neuron_names = ['neuron_' + str(i) for i in range(122)]
trials = ['trial_' + str(i) for i in range(12)]

baseline = pd.DataFrame(index = trials, columns =neuron_names)

for neuron in neuron_names :
    for i in range(12) :
        minimum = np.min(neural_data[neuron][neural_data.index.get_level_values("Trial")==i].rolling(10).min())
        baseline[neuron][trials[i]] = minimum

baseline

In [ ]:
# To see the noise, we plot a neuron zoomed in a specific interval of time

plt.plot(neural_data[neural_data.index.get_level_values("Trial")==0]['t'].iloc[0:500], neural_data['neuron_0'][neural_data.index.get_level_values("Trial")==0].iloc[0:500])
plt.xlabel("Time [s]")
plt.ylabel("Fluorescence [-]")
plt.title("Noise visualization")
plt.show()

In [ ]:
# Let's compare different sigma values for the gaussian filter (0, 3, 6)

gaussian_f1 = np.copy(neural_data["neuron_0"][neural_data.index.get_level_values("Trial")==0].iloc[0:500])
gaussian_f2 = np.copy(neural_data["neuron_0"][neural_data.index.get_level_values("Trial")==0].iloc[0:500])
gaussian_f3 = np.copy(neural_data["neuron_0"][neural_data.index.get_level_values("Trial")==0].iloc[0:500])

plt.plot(neural_data[neural_data.index.get_level_values("Trial")==0]["t"].iloc[0:500], gaussian_f1, label = "original data")
plt.plot(neural_data[neural_data.index.get_level_values("Trial")==0]["t"].iloc[0:500], gaussian_filter1d(gaussian_f2, 3), label = "Gaussian filter, σ=3")
plt.plot(neural_data[neural_data.index.get_level_values("Trial")==0]["t"].iloc[0:500], gaussian_filter1d(gaussian_f3, 6), label = "Gaussian filter, σ=6")
plt.xlabel("Time [s]")
plt.ylabel("Fluorescence [-]")
plt.title("Noise filering")
plt.legend()
plt.show()

In [ ]:
# From the plot we can see that a sigma of 6 is enough to approximate the noisy data so we
# will use this filter to denoise the data.

# Apply gaussian filter of sigma = 6 to the data
neural_data_filtered = neural_data.copy()

columns = neural_data_filtered.columns.values.tolist()
neurons = columns[1:]
for neuron in neurons : 
    for i in range(12) :
        neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i] = gaussian_filter1d(neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i], sigma=6)

In [ ]:
# Let's replot the first the non-modified data, then the denoised one

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]
plot_scaling_factor = 5000

for j in range(5):
    random_neuron = random.randrange(122)
    neuron = "neuron_"+str(random_neuron)

    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data[neural_data.index.get_level_values("Trial")==0]["t"], i*plot_scaling_factor + neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].values - neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].values[0])
        ytick.append(i*plot_scaling_factor)

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Fluorescence [-]")
    ax.set_title(f'Fluorescence data of neuron {random_neuron} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

## Neuronal data analysis

In [ ]:
neural_data_filtered
# 123 neurons
# 12 trials
# 4040 time points per trial
# 48480 total time points

In [ ]:
# neural_data :
# 123 neurons
# 12 trials
# 4040 time points per trial
# 48480 total time points

# PCA : each neuron as a feature, each time point as a sample, all trials at once

# X1 : (n_samples, n_features)
# size : (48480*123)
# 123 components
# 48480 samples

pca = PCA(n_components=123)
X1 = pca.fit_transform(neural_data_filtered.iloc[:, 1:])

In [ ]:
#Plot result
nb_trials = 12
plot_scaling_factor = 15000

for j in range(5) : # the 5 first components
    ytick = []
    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data_filtered[neural_data_filtered.index.get_level_values("Trial")==0]["t"], i*plot_scaling_factor + X1[i*4040:(i+1)*4040, j])
        ytick.append(X1[i*4040, j] + i*plot_scaling_factor)

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Component [p.d.u.]")
    ax.set_title(f"PCA component {j}")

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
# Result PCA
loadings = pca.components_
# in the loadings matrix, we have :
# each column a neuron
# each line a component

n = 5
sorted_array = np.sort(loadings, None)
print(f"loadings {loadings}")
print(f"max value {loadings.max():2.2}")
print(f"n biggest values:")
for i in range(n):
    print(f"\t{sorted_array[-1-i]:2.2}")
print(f"Biggest neurons contribs:")
for i in range(n):
    print(f"\t{np.where(loadings == sorted_array[-1-i])}")

fig, ax = plt.subplots(figsize = (16,12))
pos = ax.imshow(loadings)
fig.colorbar(pos, ax = ax)
plt.xlabel("Principal component 1")
plt.ylabel("Principal component 2")
plt.show()

In [ ]:
# Loadings are the covariances/correlations between the original variables and the unit-scaled components.
print(f"Explained variance (by component) :\n{pca.explained_variance_ratio_}")
print(f"Explained variance (cumulated) :\n{np.cumsum(pca.explained_variance_ratio_)}")

In [ ]:
# Calculate the point density of te 2 firsts components that explain 69% of the total variance
xy = np.vstack([X1[:,0], X1[:,1]])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(X1[:,0], X1[:,1], c=z, s=100)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"Density-coded scatter-plot of the first two components of the PCA results")

plt.show()

In [ ]:
# We can try to cluster the data with KMeans

model_km_mean = KMeans(n_clusters=3, random_state=0)
kmeans = model_km_mean.fit_predict(X1[:,0:2])

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(X1[:,0], X1[:,1], c=kmeans)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"k-means of PCA results")

plt.show()

In [ ]:
# Let's do the t-SNE on the PCA
tsne= TSNE(n_components=2)
Y1 = tsne.fit_transform(X1)

# Y1 :
# size : (48480*2)
# 48480 samples
# 2 components

In [ ]:
# Calculate the point density
xy = np.vstack([Y1[:,0], Y1[:,1]])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y1[:,0], Y1[:,1], c=z, s=100)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"Scatter-plot of TSNE results")

plt.show()

In [ ]:
# neural_data :
# 123 neurons
# 12 trials
# 4040 time points per trial
# 48480 total time points

# PCA : this time, each neuron as a sample, each (averaged) time point as a feature

# x2 : (n_samples, n_features)
# size : (123*12)
# 123 samples
# 12 components

neural_data_mean = np.zeros((12,123))
for i in range(12):
    neural_data_mean[i,:] = neural_data_filtered.iloc[i*4040:(i+1)*4040, 1:].mean(axis = 0)

pca = PCA(n_components=12)
X2 = pca.fit_transform(neural_data_mean.T)

In [ ]:
#Let's also try TSNE on this data 

tsne_mean = TSNE(n_components=2)
X2_mean = tsne_mean.fit_transform(neural_data_mean.T)

In [ ]:
Y2 = tsne.fit_transform(X2)

# Y2 :
# size : (123*2)
# 123 samples
# 2 components

# Calculate the point density
xy = np.vstack([Y2[:,0], Y2[:,1]])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2[:,0], Y2[:,1], c=z, s=100)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"Scatter-plot of PCA results")

plt.show()

In [ ]:
Y2_mean = tsne.fit_transform(X2_mean)

# Y2 :
# size : (123*2)
# 123 samples
# 2 components

# Calculate the point density
xy = np.vstack([Y2_mean[:,0], Y2_mean[:,1]])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2_mean[:,0], Y2_mean[:,1], c=z, s=100)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"Scatter-plot of TSNE results")

plt.show()

In [ ]:
#Here I used the TSNE components to distinguish datapoints

model_km_mean = KMeans(n_clusters=3, random_state=0)
kmeans = model_km_mean.fit_predict(Y2_mean)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2_mean[:,0], Y2_mean[:,1], c=kmeans)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"k-means of TSNE results")

plt.show()

In [ ]:
# With the result above we can plot neural trace depending on group 

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]

for i in np.where(model_km_mean.labels_ == 2)[0] :
    neuron = 'neuron_'+str(i)
    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data_filtered[neural_data_filtered.index.get_level_values("Trial")==0]['t'], i*2500 + neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].values, label = neuron)
        ytick.append(neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].iloc[0] + i*2500)
    ax.set_ylabel('Fluorescence (a.u)')
    ax.set_xlabel('Time (sec)')
    ax.set_title(f'Fluorescence data of {neuron} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
# With the result above we can plot neural trace depending on group 

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]

for i in np.where(model_km_mean.labels_ == 0)[0] :
    neuron = 'neuron_'+str(i)
    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data_filtered[neural_data_filtered.index.get_level_values("Trial")==0]['t'], i*2500 + neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].values, label = neuron)
        ytick.append(neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].iloc[0] + i*2500)
    ax.set_ylabel('Fluorescence (a.u)')
    ax.set_xlabel('Time (sec)')
    ax.set_title(f'Fluorescence data of {neuron} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
# With the result above we can plot neural trace depending on group 

nb_trials = 12
trials_name = ['trial_' + str(i+1) for i in range(nb_trials)]

for i in np.where(model_km_mean.labels_ == 1)[0] :
    neuron = 'neuron_'+str(i)
    ytick = []

    fig, ax = plt.subplots(figsize = (16,6))

    for i in range(nb_trials):   
        ax.plot(neural_data_filtered[neural_data_filtered.index.get_level_values("Trial")==0]['t'], i*2500 + neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].values, label = neuron)
        ytick.append(neural_data_filtered[neuron][neural_data_filtered.index.get_level_values("Trial")==i].iloc[0] + i*2500)
    ax.set_ylabel('Fluorescence (a.u)')
    ax.set_xlabel('Time (sec)')
    ax.set_title(f'Fluorescence data of {neuron} across trials')

    plt.yticks(np.array(ytick), trials_name)
    plt.grid()
    plt.show()

In [ ]:
# We can see if GMM clusters the data correctly
model_gm_mean = GaussianMixture(n_components=3, random_state=0)
gm_mean = model_gm_mean.fit_predict(Y2_mean)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2_mean[:,0], Y2_mean[:,1], c=gm_mean)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"GMM of TSNE results")

plt.show()

In [ ]:
# We can see if KMeans clusters the data correctly
model_km = KMeans(n_clusters=3, random_state=0)
kmeans = model_km.fit_predict(Y2)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2[:,0], Y2[:,1], c=kmeans)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"k-means of PCA results")

plt.show()

In [ ]:
# We can see if GMM clusters the data correctly
model_gm = GaussianMixture(n_components=3, random_state=0)
gm = model_gm.fit_predict(Y2)

fig, ax = plt.subplots(figsize = (16,12))

ax.scatter(Y2[:,0], Y2[:,1], c=gm)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"GMM of PCA results")

plt.show()

In [ ]:
# Let's compare whether Kmeans on mean data and normal data gives same result
comparison = model_km_mean.labels_ == gm_mean
indices = [i for i, x in enumerate(comparison.tolist()) if x == False]
indices #neurons that are differently classified between the 2 methods

In [ ]:
# Let's compare whether GMM on mean data and normal data gives same result
comparison = model_km.labels_ == gm
indices = [i for i, x in enumerate(comparison.tolist()) if x == False]
indices #neurons that are differently classified between the 2 methods

## Identifying features of behaviour

In [ ]:
# Extract data joint
behav_data_joint = behav_data.filter(regex="joint")
# Extract data angle
behav_data_angle = behav_data.filter(regex="angle")
# Extract by trial
nb_trials = 12
nb_time = behav_data_joint[behav_data.index.get_level_values("Trial")==1].shape[0]
nb_feature_joint = behav_data_joint[behav_data.index.get_level_values("Trial")==1].shape[1]
nb_feature_angle = behav_data_angle[behav_data.index.get_level_values("Trial")==1].shape[1]

print(f'Nb time : {nb_time}, Nb feature joint : {nb_feature_joint}, Nb feature angle : {nb_feature_angle}')

joint_data = np.zeros((nb_trials, nb_time, nb_feature_joint))
angle_data = np.zeros((nb_trials, nb_time, nb_feature_angle))
for i in range(nb_trials):
    joint_data[i,:,:] = behav_data_joint[behav_data.index.get_level_values("Trial")==i].to_numpy()
    angle_data[i,:,:] = behav_data_angle[behav_data.index.get_level_values("Trial")==i].to_numpy()


### PCA

In [ ]:
# Compute PCA for joint pose
nb_components_joint = 13
output_pca_joint = np.zeros((nb_trials, nb_time, nb_components_joint))
for i in range(nb_trials):
    pca_joint = PCA(n_components=nb_components_joint)
    output_pca_joint[i,:,:] = pca_joint.fit_transform(joint_data[i,:,:])
    print(f"Trail {i}, Nb components : {nb_components_joint}, Variance : {sum(pca_joint.explained_variance_ratio_)}")

In [ ]:
# Compute PCA for joint angle
nb_components_angle = 17
output_pca_angle = np.zeros((nb_trials, nb_time, nb_components_angle))
for i in range(nb_trials):
    pca_joint = PCA(n_components=nb_components_angle)
    output_pca_angle[i,:,:] = pca_joint.fit_transform(angle_data[i,:,:])
    print(f"Trail {i}, Nb components : {nb_components_angle}, Variance : {sum(pca_joint.explained_variance_ratio_)}")

### Wavelet transform

In [ ]:
# Wavelet for Joint
# Parameters
nb_freqs=25
sample_frequency = 100
min_frequency = 1
max_frequency = 50

# Result matrix
freqs_joint = np.zeros((nb_trials, nb_freqs))
power_joint = np.zeros((nb_trials, nb_time))
wavelet_joint = np.zeros((nb_trials, nb_time, nb_components_joint*nb_freqs))

# Compute Wavelet
for i in range(nb_trials):
    print(f'Joint : Trial {i+1} / {nb_trials}')
    temp1, temp2, temp3 = wavelet_transform(output_pca_joint[i,:,:], n_freqs=nb_freqs, fsample=sample_frequency, fmin=min_frequency, fmax=max_frequency)
    freqs_joint[i,:] = temp1
    power_joint[i,:] = temp2
    wavelet_joint[i,:,:] = temp3

In [ ]:
# Wavelet for Angle
# Parameters
nb_freqs=25
sample_frequency = 100
min_frequency = 1
max_frequency = 50

# Result matrix
freqs_angle = np.zeros((nb_trials, nb_freqs))
power_angle = np.zeros((nb_trials, nb_time))
wavelet_angle = np.zeros((nb_trials, nb_time, nb_components_angle*nb_freqs))

for i in range(nb_trials):
    print(f'Angle : Trial {i+1} / {nb_trials}')
    temp4, temp5, temp6 = wavelet_transform(output_pca_angle[i,:,:], n_freqs=nb_freqs, fsample=sample_frequency, fmin=min_frequency, fmax=max_frequency)
    freqs_angle[i,:] = temp4
    power_angle[i,:] = temp5
    wavelet_angle[i,:,:] = temp6

### t-SNE

In [ ]:
# t-SNE for joint form PCA
nb_components_tsne = 2
tsne_joint_pca = np.zeros((nb_trials, nb_time, nb_components_tsne))
for i in range(nb_trials):
    print(f'Joint : Trial {i+1} / {nb_trials}')
    tsne = TSNE(n_components=nb_components_tsne)
    tsne_joint_pca[i,:,:] = tsne.fit_transform(output_pca_joint[i,:,:])

In [ ]:
# t-SNE for joint form wavelete
nb_components_tsne = 2
tsne_joint_wavelete = np.zeros((nb_trials, nb_time, nb_components_tsne))
for i in range(nb_trials):
    print(f'Joint : Trial {i+1} / {nb_trials}')
    tsne = TSNE(n_components=nb_components_tsne)
    tsne_joint_wavelete[i,:,:] = tsne.fit_transform(wavelet_joint[i,:,:])

In [ ]:
# t-SNE for angle form PCA
nb_components_tsne = 2
tsne_angle_pca = np.zeros((nb_trials, nb_time, nb_components_tsne))
for i in range(nb_trials):
    print(f'Joint : Trial {i+1} / {nb_trials}')
    tsne = TSNE(n_components=nb_components_tsne)
    tsne_angle_pca[i,:,:] = tsne.fit_transform(output_pca_angle[i,:,:])

In [ ]:
# t-SNE for angle form wavelete
nb_components_tsne = 2
tsne_angle_wavelete = np.zeros((nb_trials, nb_time, nb_components_tsne))
for i in range(nb_trials):
    print(f'Joint : Trial {i+1} / {nb_trials}')
    tsne = TSNE(n_components=nb_components_tsne)
    tsne_angle_wavelete[i,:,:] = tsne.fit_transform(wavelet_angle[i,:,:])

### Plot result
#### Joint
From PCA -> Wavelet, PCA -> t-SNE, PCA -> Wavelet -> t-SNE

In [ ]:
# Output Wavelete for Joint
fig, ax = plt.subplots(nrows = nb_trials, ncols=1 ,figsize = (12,100), sharex = True)
for nb in range(nb_trials):
    ax[nb].imshow(wavelet_joint[nb,:,:].T,aspect=30)
    ax[nb].set_ylabel('Component')
    ax[nb].set_xlabel('time')
    ax[nb].set_title(f'Trial {nb}')

In [ ]:
# Plot form t-SNE for joint form PCA
for i in range(nb_trials):
    # Peform the kernel density estimate
    x = tsne_joint_pca[i,:,0]
    y = tsne_joint_pca[i,:,1]
    values = np.vstack([x, y])
    z = st.gaussian_kde(values)(values)

    # Plot
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.scatter(x,y,c=z, s=100)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} : 2D Gaussian Kernel density estimation')

In [ ]:
# Plot form t-SNE for joint form wavelete
for i in range(nb_trials):
    # Peform the kernel density estimate
    x = tsne_joint_wavelete[i,:,0]
    y = tsne_joint_wavelete[i,:,1]
    values = np.vstack([x, y])
    z = st.gaussian_kde(values)(values)

    # Plot
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.scatter(x,y,c=z, s=100)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} : 2D Gaussian Kernel density estimation')

#### Angle
From PCA -> Wavelet, PCA -> t-SNE, PCA -> Wavelet -> t-SNE

In [ ]:
# Output Wavelete for angle
fig, ax = plt.subplots(nrows = nb_trials, ncols=1 ,figsize = (12,100), sharex = True)
for nb in range(nb_trials):
    ax[nb].imshow(wavelet_angle[nb,:,:].T,aspect=30)
    ax[nb].set_ylabel('Component')
    ax[nb].set_xlabel('time')
    ax[nb].set_title(f'Trial {nb}')

In [ ]:
# Plot form t-SNE for angle form PCA
for i in range(nb_trials):
    # Peform the kernel density estimate
    x = tsne_angle_pca[i,:,0]
    y = tsne_angle_pca[i,:,1]
    values = np.vstack([x, y])
    z = st.gaussian_kde(values)(values)

    # Plot
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.scatter(x,y,c=z, s=100)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} : 2D Gaussian Kernel density estimation')

In [ ]:
# Plot form t-SNE for angle form wavelete
for i in range(nb_trials):
    # Peform the kernel density estimate
    x = tsne_angle_wavelete[i,:,0]
    y = tsne_angle_wavelete[i,:,1]
    values = np.vstack([x, y])
    z = st.gaussian_kde(values)(values)

    # Plot
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.scatter(x,y,c=z, s=100)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} : 2D Gaussian Kernel density estimation')

In [ ]:
# We can try to plot different timepoints in different color
for i in range(nb_trials):
    # Peform the kernel density estimate
    x = tsne_angle_pca[i,:,0]
    y = tsne_angle_pca[i,:,1]
    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)

    # Plot
    fig = plt.figure(figsize=(8,8))
    colors = cm.rainbow(np.linspace(0, 1, len(x)))
    plt.scatter(x, y, color=colors)
    #ax.set_xlim(xmin, xmax)
    #ax.set_ylim(ymin, ymax)
    #cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
    #ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
    #cset = ax.contour(xx, yy, f, colors='k')
    #ax.clabel(cset, inline=1, fontsize=10)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'Trial {i} : 2D Gaussian Kernel density estimation')
    break

# Part 2.2

In [ ]:
#Let's first look at our labels 
labels_unique = behav_data['Manual'].unique()
print(labels_unique)

In [ ]:
#Let's look whether they are different from the predictions one
comparison = behav_data['Manual'] == behav_data['Prediction']
print(f'We have {len([i for i, x in enumerate(comparison.tolist()) if x == True])} behavioral match between predictions and manual labeling')
print(f'And {len([i for i, x in enumerate(comparison.tolist()) if x == False])} behavioral differences between predictions and manual labeling')

indices = [i for i, x in enumerate(comparison.tolist()) if x == False]
names = []
count = []
for indice in indices : 
    pred = behav_data.iloc[indice]['Prediction']
    man = behav_data.iloc[indice]['Manual']
    if pred+"/"+man in names : 
        count[names.index(pred+"/"+man)] = count[names.index(pred+"/"+man)] + 1
    else : 
        names.append(pred+"/"+man)
        count.append(0)

In [ ]:
# The differences between manual and prediction labels
for i in range(len(names)) :
    print(names[i], '->' ,count[i])

In [ ]:
#Let's first plot the tsne joint PCA
for i in range(nb_trials):
    x = tsne_joint_pca[i,:,0]
    y = tsne_joint_pca[i,:,1]
    
    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    plt.legend()
    plt.show()

In [ ]:
# Then the tsne of joint wavelete 
for i in range(nb_trials):
    x = tsne_joint_wavelete[i,:,0]
    y = tsne_joint_wavelete[i,:,1]
    
    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    plt.legend()
    plt.show()

In [ ]:
# Then the tsne of angle pca 
for i in range(nb_trials):
    x = tsne_angle_pca[i,:,0]
    y = tsne_angle_pca[i,:,1]
    
    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    plt.legend()
    plt.show()

In [ ]:
# Then the tsne of angle wavelete
for i in range(nb_trials):
    x = tsne_angle_wavelete[i,:,0]
    y = tsne_angle_wavelete[i,:,1]
    
    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    plt.legend()
    plt.show()

In [ ]:
def draw_ellipse(gmm, ax) : 
    colors = ['black', 'red', 'blue', 'green', 'purple']
    for n, color in enumerate(colors): 
        covariances = gmm.covariances_[n][:2, :2]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                      180 + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        ax.set_aspect('equal', 'datalim')

In [ ]:
#Now let's try to cluster the data with a Gaussian mixture model as KMeans would not cluster circle-shaped data.
# We use a mixture of 5 components as we have 5 categories 

for i in range(nb_trials):
    x = tsne_joint_pca[i,:,0]
    y = tsne_joint_pca[i,:,1]

    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)

    gmm = GaussianMixture(n_components=5, max_iter=100)
    x_label = gmm.fit_predict(tsne_joint_pca[i,:])
    #print(gmm.means_)
    #print('then')
    #print(gmm.covariances_)
    #print("Score : ")
    #print(gmm.score(tsne_joint_pca[i,:]))
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    draw_ellipse(gmm, ax)
    plt.legend()
    plt.show()

In [ ]:
#Now let's try to cluster the data with a KMeans.
# We use a mixture of 5 components as we have 5 categories 

x = tsne_angle_wavelete[11,:,0]
y = tsne_angle_wavelete[11,:,1]

xmin, xmax = min(x), max(x)
ymin, ymax = min(y), max(y)
    
labels = behav_data[behav_data.index.get_level_values("Trial")==11]['Manual']
dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
label = labels.replace(dic_colors)

kmeans = KMeans(n_clusters=5, max_iter=100)
x_label = kmeans.fit_predict(tsne_angle_wavelete[11,:])
    
fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title(f'Trial {i} :n')
    
#plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
plt.scatter(x,y, s=0.5, c=x_label, label=dic_colors)
plt.show()

In [ ]:
# From : https://medium.com/@plog397/functions-to-plot-kmeans-hierarchical-and-dbscan-clustering-c4146ed69744

def dbscan(X, eps, min_samples):
    ss = StandardScaler()
    X = ss.fit_transform(X)
    db = DBSCAN(eps=eps, min_samples=min_samples)
    db.fit(X)
    y_pred = db.fit_predict(X)
    plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired', alpha=0.5)
    plt.title("DBSCAN")

In [ ]:
# We can clearly see that Kmeans does not properly cluster the data, thus we can try to use 
# Density-based spatial clustering (DBSCAN) that is a better method for nonlinear clusters. 

for i in range(nb_trials):
    x = tsne_angle_wavelete[i,:,0]
    y = tsne_angle_wavelete[i,:,1]

    xmin, xmax = min(x), max(x)
    ymin, ymax = min(y), max(y)
    
    labels = behav_data[behav_data.index.get_level_values("Trial")==i]['Manual']
    dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple'}
    label = labels.replace(dic_colors)
    
    dbscan(tsne_angle_wavelete[i,:], 0.1, 1)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title(f'Trial {i} :n')
    plt.scatter(x,y, s=0.5, c=label.values, label=dic_colors)
    plt.legend()
    plt.show()

# Part 2.3

In [ ]:
# Let's use all joint angles, 25 wavelet features, 1 to 50 Hz wavelet, no PCA

# Parameters
nb_freqs=25
sample_frequency = 100
min_frequency = 1
max_frequency = 50  
nb_components_angle = 42
    
# Angle
freqs_angle = np.zeros((nb_trials, nb_freqs))
power_angle = np.zeros((nb_trials, nb_time))
wavelet_angle = np.zeros((nb_trials, nb_time, nb_components_angle*nb_freqs))

for i in range(nb_trials):
    print(f'Angle : Trial {i+1} / {nb_trials}')
    temp4, temp5, temp6 = wavelet_transform(angle_data[i,:,:], n_freqs=nb_freqs, fsample=sample_frequency, fmin=min_frequency, fmax=max_frequency)
    freqs_angle[i,:] = temp4
    power_angle[i,:] = temp5
    wavelet_angle[i,:,:] = temp6

In [ ]:
#Then train with random forest on the data that we split first between train and test set
X = np.reshape(wavelet_angle, (302400, 1050))
y = behav_data['Manual']

rfc = RandomForestClassifier(n_estimators=70, oob_score=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
rfc.fit(X_train, y_train)
y_prediction = rfc.predict(X_test)

In [ ]:
#Let's look at the accuracy of manual prediction
y_train_pred = rfc.predict(X_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print('Score:', rfc.score(X_test, y_test))

In [ ]:
#And then the prediction from predicted labeling
y_pred=rfc.predict(X)
y=behav_data['Prediction']
print("Accuracy:",metrics.accuracy_score(y, y_pred))

In [ ]:
# Let's look at the most important predicting features of the classifier
print(rfc.feature_importances_.argsort()[0:10]%25)
print(behav_data_angle[behav_data_angle.columns[23]])
print(behav_data_angle[behav_data_angle.columns[22]])
print(behav_data_angle[behav_data_angle.columns[24]])

In [ ]:
# And the number of differences between the manual labeling and our prediction

y_differences = rfc.predict(X)
comparison = y == y_differences
comparison_data = behav_data.iloc[[i for i, x in enumerate(comparison.tolist()) if x == False]]

comparison_2 = comparison_data['Manual'] == comparison_data['Prediction']
print(f'We have {len([i for i, x in enumerate(comparison_2.tolist()) if x == True])} behavioral match between predictions and manual labeling')
print(f'And {len([i for i, x in enumerate(comparison_2.tolist()) if x == False])} behavioral differences between predictions and manual labeling')

In [ ]:
#Let's to compare predicted data now : 
y_pred = behav_data['Prediction']
y_pred_prediction = rfc.predict(X)
print("Accuracy:",metrics.accuracy_score(y_pred, y_pred_prediction))

comparison = y_pred == y_pred_prediction
comparison_data = behav_data.iloc[[i for i, x in enumerate(comparison.tolist()) if x == False]]

comparison_2 = comparison_data['Manual'] == comparison_data['Prediction']
print(f'We have {len([i for i, x in enumerate(comparison_2.tolist()) if x == True])} behavioral match between predictions and manual labeling')
print(f'And {len([i for i, x in enumerate(comparison_2.tolist()) if x == False])} behavioral differences between predictions and manual labeling')

## Combining neural & behavioural data

### Downsampling the behavioral data 

In [ ]:
#Functions to down-sample the data (from Braun's notebook)

# these two functions are just wrappers around the numpy functions to apply them across dimension 0 only
def reduce_mean(values):
    return np.mean(values, axis=0)
def reduce_std(values):
    return np.std(values, axis=0)
def reduce_behaviour(values):
    """
    this is just a sketch for how to reduce behavioural classes. 
    It picks whatever behaviour occurs the most.
    Try to make this more stable, for example by handling the case when two behaviours are equally likely.
    You might also want to include a certainty threshold, 
    e.g. 3/4 of the behaviour frames have to be labelled the same way, otherwise it is None and the data is excluded
    """
    unique_values, N_per_unique = np.unique(values, return_counts=True)
    i_max = np.argmax(N_per_unique)
    return unique_values[i_max]

def reduce_during_2p_frame(twop_index, values, function=reduce_mean):
    """
    Reduces all values occuring during the acquisition of a
    two-photon imaging frame to a single value using the `function` given by the user.
    Parameters
    ----------
    twop_index : numpy array
        1d array holding frame indices of one trial.
    values : numpy array
        Values upsampled to the frequency of ThorSync,
        i.e. 1D numpy array of the same length as
        `frame_counter` or 2D numpy array of the same length.
    function : function
        Function used to reduce the value,
        e.g. np.mean for 1D variables
    Returns
    -------
    reduced : numpy array
        Numpy array with value for each two-photon imaging frame.
    """
    
    if len(twop_index) != len(values):
        raise ValueError("twop_index and values need to have the same length.")
    if len(values.shape) == 1:
        values = np.expand_dims(values, axis=1)
        squeeze = True
    else:
        squeeze = False
    N_samples, N_variables = values.shape
    
    index_unique = np.unique(twop_index)
    index_unique = np.delete(index_unique, index_unique==-9223372036854775808)
    
    dtype = values.dtype
    if np.issubdtype(dtype, np.number):
        dtype = np.float
    else:
        dtype = np.object
    reduced = np.empty((len(index_unique), N_variables), dtype=dtype)

    for i, index in enumerate(index_unique):
        reduced[i] = function(values[twop_index==index, :])

    return np.squeeze(reduced) if squeeze else reduced

def plot_neuron_data(neuron_list, nb_plot):
    for k in range(nb_plot):
        neuron_label = neuron_list[k]
        fig, ax = plt.subplots(6,2, figsize=(15, 15))
        fig.suptitle(f'Plot for neuron {neuron_label}')
        for i in range(2):
            for j in range(6):
                idx = j+(6*i)
                # 28, 62, 93
                neuron = 100*(neural_df_s.loc[(210301, "J1xCI9", 1, idx), "neuron_" + str(neuron_label)]-610)/610

                ax[j,i].plot(neural_df_s.loc[(210301, "J1xCI9", 1, 0), "t"], neuron, label=("neuron_" + str(neuron_label)))
                ax[j,i].fill_between(neural_df_s.loc[(210301, "J1xCI9", 1, 0), "t"], find_behavior(neural_df_s[idx*4040:idx*4040+4040], 'walking'),alpha = 0.5,label="walking")
                ax[j,i].fill_between(neural_df_s.loc[(210301, "J1xCI9", 1, 0), "t"], find_behavior(neural_df_s[idx*4040:idx*4040+4040], 'resting'), alpha = 0.5,label="resting")
                ax[j,i].set_xlabel("t (s)")
                ax[j,i].set_ylabel("$\% \Delta F/F$")
                ax[j,i].legend()

                ax[j,i].spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

                # turn off the right spine/ticks
                ax[j,i].spines['right'].set_color('none')
                ax[j,i].yaxis.tick_left()

                # set the y-spine
                ax[j,i].spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

                # turn off the top spine/ticks
                ax[j,i].spines['top'].set_color('none')
                ax[j,i].xaxis.tick_bottom()
                ax[j,i].set_title(f'Trial {idx}')
    

In [ ]:
#Let's now down-sample our data first by only down-sampling the numerical data, and only then the categorical ones
# We will create a dataframe per trial here and then concatenate to a single dataframe
columns_numerical = behav_data.columns.to_list()
columns_numerical.remove('Prediction')
columns_numerical.remove('Manual')
columns_numerical.remove('twop_index')

columns_categorical = ['Prediction', 'Manual']
dfs = []
for i in range(12) :
    twop_index = behav_data.loc[(210301, "J1xCI9", 1, i), "twop_index"].to_numpy()
    values_numerical = behav_data.loc[(210301, "J1xCI9", 1, i)][columns_numerical].to_numpy()
    reduced_numerical = reduce_during_2p_frame(twop_index, values_numerical, function=reduce_mean)

    values_categorical = behav_data.loc[(210301, "J1xCI9", 1, i)][columns_categorical].to_numpy()
    reduced_categorical = reduce_during_2p_frame(twop_index, values_categorical, function=reduce_behaviour)
    df = pd.DataFrame(reduced_numerical, columns = columns_numerical)
    df[columns_categorical] = reduced_categorical
    dfs.append(df)

In [ ]:
#Let's now put all that in a single dataframe
behav_data_r = pd.concat(dfs, keys=[0,1,2,3,4,5,6,7,8,9,10,11], axis=0, names=['Trial', 'Frame'])
behav_data_r

###  Identifying correlations of individual neurons

In [ ]:
#Let's first divide the behavioral data by states 

states = behav_data_r['Manual'].unique()
print(f'Possible states are :{states}')

resting_state = behav_data_r.loc[behav_data_r['Manual'] == 'resting']
print(f'Resting state df length : {len(resting_state)}')
walking_state = behav_data_r.loc[behav_data_r['Manual'] == 'walking']
print(f'Walking state df length : {len(walking_state)}')
anterior_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'anterior_grooming']
print(f'Anterior grooming state df length : {len(anterior_grooming_state)}')
antennal_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'antennal_grooming']
print(f'Antennal grooming state df length : {len(antennal_grooming_state)}')
foreleg_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'foreleg_grooming']
print(f'Foreleg grooming state df length : {len(foreleg_grooming_state)}')
abdominal_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'abdominal_grooming']
print(f'Abdominal grooming state df length : {len(abdominal_grooming_state)}')
abdominal_pushing_state = behav_data_r.loc[behav_data_r['Manual'] == 'abdominal_pushing']
print(f'Abdominal pushing state df length : {len(abdominal_pushing_state)}')
posterior_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'posterior_grooming']
print(f'Posterior grooming state df length : {len(posterior_grooming_state)}')
hindleg_grooming_state = behav_data_r.loc[behav_data_r['Manual'] == 'hindleg_grooming']
print(f'Hindleg grooming state df length : {len(hindleg_grooming_state)}')

In [ ]:
# Let's now look at mean neuron activity during behavior
resting_means = []
walking_means = []
anterior_grooming_means = []
antennal_grooming_means= []
foreleg_grooming_means=[]
abdominal_grooming_means = []
abdominal_pushing_means = []
posterior_grooming_means = []
hindleg_grooming_means = []

for neuron in neural_data_filtered.columns :
    if neuron =='t' : pass
    else : 
        
        resting_mean = np.mean(neural_data_filtered.iloc[resting_state.index.get_level_values(level=1)][neuron])
        resting_means.append(resting_mean)
        walking_mean = np.mean(neural_data_filtered.iloc[walking_state.index.get_level_values(level=1)][neuron])
        walking_means.append(walking_mean)
        anterior_grooming_mean = np.mean(neural_data_filtered.iloc[anterior_grooming_state.index.get_level_values(level=1)][neuron])
        anterior_grooming_means.append(anterior_grooming_mean)
        antennal_grooming_mean = np.mean(neural_data_filtered.iloc[antennal_grooming_state.index.get_level_values(level=1)][neuron])
        antennal_grooming_means.append(antennal_grooming_mean)
        foreleg_grooming_mean = np.mean(neural_data_filtered.iloc[foreleg_grooming_state.index.get_level_values(level=1)][neuron])
        foreleg_grooming_means.append(foreleg_grooming_mean)
        abdominal_grooming_mean = np.mean(neural_data_filtered.iloc[abdominal_grooming_state.index.get_level_values(level=1)][neuron])
        abdominal_grooming_means.append(abdominal_grooming_mean)
        abdominal_pushing_mean = np.mean(neural_data_filtered.iloc[abdominal_pushing_state.index.get_level_values(level=1)][neuron])
        abdominal_pushing_means.append(abdominal_pushing_mean)
        posterior_grooming_mean = np.mean(neural_data_filtered.iloc[posterior_grooming_state.index.get_level_values(level=1)][neuron])
        posterior_grooming_means.append(posterior_grooming_mean)
        hindleg_grooming_mean = np.mean(neural_data_filtered.iloc[hindleg_grooming_state.index.get_level_values(level=1)][neuron])
        hindleg_grooming_means.append(hindleg_grooming_mean)

plt.figure(figsize=(16,10))
plt.boxplot([resting_means, walking_means, anterior_grooming_means,antennal_grooming_means, foreleg_grooming_means, \
             abdominal_grooming_means,abdominal_pushing_means,posterior_grooming_means,hindleg_grooming_means])
plt.xticks([1, 2, 3, 4, 5, 6, 7,8,9], states.tolist())
plt.ylabel("mean $\% \Delta F/F$ neuron")
plt.show()

In [ ]:
#LEt's plot a matrix with the neurons and their value for behavior
plt.figure(figsize=(16,10))
plt.matshow([resting_means, walking_means, anterior_grooming_means,antennal_grooming_means,foreleg_grooming_means, \
             abdominal_grooming_means,abdominal_pushing_means,posterior_grooming_means,hindleg_grooming_means], fignum=1, aspect='auto')
plt.yticks([0, 1, 2, 3, 4, 5, 6,7,8], states.tolist())
plt.xlabel("neurons number")
plt.show()

In [ ]:
#Let's now try to standardize the data before plotting
neural_df_s = neural_data_filtered.copy()

for neuron in neural_data_filtered.columns :
    if neuron =='t' : pass
    else : 
        neural_df_s[neuron] = zscore(neural_data_filtered[neuron])
        
neural_df_s

In [ ]:
#To test the difference of means we need to add a column which is the manual behavior to the neural dataframe
neural_df_s['Manual'] = behav_data_r['Manual'].values
neural_df_s = neural_df_s.astype({"Manual":str})
neural_df_s

In [ ]:
#now that we have this dataset we can plot behaviors from neuronal analysis
fig, ax = plt.subplots(figsize = (16,12))

labels = neural_df_s['Manual']
dic_colors = {'resting': 'black', 'walking': 'red', 'anterior_grooming':'blue', 'abdominal_pushing':'green', 'posterior_grooming':'purple', 'antennal_grooming' : 'yellow', 'foreleg_grooming' : 'pink', 'abdominal_grooming':'grey', 'hindleg_grooming' : 'brown'}
label = labels.replace(dic_colors)
    
ax.scatter(X1[:,0], X1[:,1], s=100, c=label.values, label=dic_colors)
ax.set_xlabel("Component 0 [p.d.u.]")
ax.set_ylabel("Component 1 [p.d.u.]")
ax.set_title(f"Density-coded scatter-plot of the first two components of the PCA results")
plt.legend()
plt.savefig('figures/pca_2_neuron_label.eps', format='eps')

plt.show()

In [ ]:
# Let's now look at mean neuron activity during behavior
resting_means = []
walking_means = []
anterior_grooming_means = []
antennal_grooming_means= []
foreleg_grooming_means=[]
abdominal_grooming_means = []
abdominal_pushing_means = []
posterior_grooming_means = []
hindleg_grooming_means = []
resting_stds = []
walking_stds = []
anterior_grooming_stds = []
antennal_grooming_stds= []
foreleg_grooming_stds=[]
abdominal_grooming_stds = []
abdominal_pushing_stds = []
posterior_grooming_stds = []
hindleg_grooming_stds = []

# And we will test whether each neuron has a significant mean difference in activation depending on the state
tukey_test = []

for neuron in neural_df_s.columns :
    if neuron =='t' or neuron =='Manual' : pass
    else : 
        #ANOVA + TUKEY test in a new dataframe
        formula = f'{neuron}~C(Manual)'
        model = ols(formula, data=neural_df_s).fit()
        aov_table = anova_lm(model)
        if aov_table['PR(>F)'][0] <0.0005 : 
            comparison = MultiComparison(neural_df_s[neuron], neural_df_s['Manual'])
            comparison_results = comparison.tukeyhsd()
            tukey_test.append(comparison_results.reject)
        
        resting_mean = np.mean(neural_df_s.iloc[resting_state.index.get_level_values(level=1)][neuron])
        resting_std= np.std(neural_df_s.iloc[resting_state.index.get_level_values(level=1)][neuron])
        resting_means.append(resting_mean)
        resting_stds.append(resting_std)
        walking_mean = np.mean(neural_df_s.iloc[walking_state.index.get_level_values(level=1)][neuron])
        walking_std= np.std(neural_df_s.iloc[walking_state.index.get_level_values(level=1)][neuron])
        walking_means.append(walking_mean)
        walking_stds.append(walking_std)
        anterior_grooming_mean = np.mean(neural_df_s.iloc[anterior_grooming_state.index.get_level_values(level=1)][neuron])
        anterior_grooming_std = np.std(neural_df_s.iloc[anterior_grooming_state.index.get_level_values(level=1)][neuron])
        anterior_grooming_means.append(anterior_grooming_mean)
        anterior_grooming_stds.append(anterior_grooming_std)
        antennal_grooming_mean = np.mean(neural_df_s.iloc[antennal_grooming_state.index.get_level_values(level=1)][neuron])
        antennal_grooming_std = np.std(neural_df_s.iloc[antennal_grooming_state.index.get_level_values(level=1)][neuron])
        antennal_grooming_means.append(antennal_grooming_mean)
        antennal_grooming_stds.append(antennal_grooming_std)
        foreleg_grooming_mean = np.mean(neural_df_s.iloc[foreleg_grooming_state.index.get_level_values(level=1)][neuron])
        foreleg_grooming_std = np.std(neural_df_s.iloc[foreleg_grooming_state.index.get_level_values(level=1)][neuron])
        foreleg_grooming_means.append(foreleg_grooming_mean)
        foreleg_grooming_stds.append(foreleg_grooming_std)
        abdominal_grooming_mean = np.mean(neural_df_s.iloc[abdominal_grooming_state.index.get_level_values(level=1)][neuron])
        abdominal_grooming_std = np.std(neural_df_s.iloc[abdominal_grooming_state.index.get_level_values(level=1)][neuron])
        abdominal_grooming_means.append(abdominal_grooming_mean)
        abdominal_grooming_stds.append(abdominal_grooming_std)
        abdominal_pushing_mean = np.mean(neural_df_s.iloc[abdominal_pushing_state.index.get_level_values(level=1)][neuron])
        abdominal_pushing_std = np.std(neural_df_s.iloc[abdominal_pushing_state.index.get_level_values(level=1)][neuron])
        abdominal_pushing_means.append(abdominal_pushing_mean)
        abdominal_pushing_stds.append(abdominal_pushing_std)
        posterior_grooming_mean = np.mean(neural_df_s.iloc[posterior_grooming_state.index.get_level_values(level=1)][neuron])
        posterior_grooming_std = np.std(neural_df_s.iloc[posterior_grooming_state.index.get_level_values(level=1)][neuron])
        posterior_grooming_means.append(posterior_grooming_mean)
        posterior_grooming_stds.append(posterior_grooming_std)
        hindleg_grooming_mean = np.mean(neural_df_s.iloc[hindleg_grooming_state.index.get_level_values(level=1)][neuron])
        hindleg_grooming_std = np.std(neural_df_s.iloc[hindleg_grooming_state.index.get_level_values(level=1)][neuron])
        hindleg_grooming_means.append(hindleg_grooming_mean)
        hindleg_grooming_stds.append(hindleg_grooming_std)
        
plt.figure(figsize=(16,10))
plt.boxplot([resting_means, walking_means, anterior_grooming_means,antennal_grooming_means, foreleg_grooming_means, \
             abdominal_grooming_means,abdominal_pushing_means,posterior_grooming_means,hindleg_grooming_means])
plt.xticks([1, 2, 3, 4, 5, 6, 7,8,9], states.tolist())
plt.ylabel("mean $\% \Delta F/F$ neuron")
plt.show()

In [ ]:
#We will also create a DataFrame with the results of the Tukey test
columns = ['ab_g', 'ab_p', 'al_g', 'ar_g', 'f_g', 'h_g', 'p_g', 'r', 'w']
tukey_df = pd.DataFrame(data = tukey_test, columns=list(combinations(columns, 2)))
"""    
                                                                        reject
A results of ’reject = True’ means that a significant difference has been observed.
abdominal_grooming  abdominal_pushing   0.1539    0.9 -0.2159  0.5238  False
abdominal_grooming  antennal_grooming    0.267 0.5477 -0.1508  0.6849  False
abdominal_grooming  anterior_grooming   0.0441    0.9 -0.3184  0.4067  False
abdominal_grooming   foreleg_grooming   0.4392 0.0145  0.0483    0.83   True
abdominal_grooming   hindleg_grooming  -0.2284    0.9 -2.1849   1.728  False
abdominal_grooming posterior_grooming   0.4429 0.0138  0.0504  0.8354   True
abdominal_grooming            resting  -0.5252  0.001 -0.8859 -0.1644   True
abdominal_grooming            walking   0.5014  0.001  0.1408  0.8621   True
 abdominal_pushing  antennal_grooming   0.1131 0.8156 -0.1146  0.3408  False
 abdominal_pushing  anterior_grooming  -0.1098 0.0081 -0.2032 -0.0165   True
 abdominal_pushing   foreleg_grooming   0.2852  0.001  0.1118  0.4586   True
 abdominal_pushing   hindleg_grooming  -0.3824    0.9 -2.3072  1.5424  False
 abdominal_pushing posterior_grooming    0.289  0.001  0.1119   0.466   True
 abdominal_pushing            resting  -0.6791  0.001 -0.7651 -0.5931   True
 abdominal_pushing            walking   0.3475  0.001  0.2617  0.4333   True
 antennal_grooming  anterior_grooming  -0.2229 0.0364 -0.4386 -0.0073   True
 antennal_grooming   foreleg_grooming   0.1721 0.5071 -0.0884  0.4326  False
 antennal_grooming   hindleg_grooming  -0.4955    0.9 -2.4301  1.4391  False
 antennal_grooming posterior_grooming   0.1758 0.4917 -0.0871  0.4388  False
 antennal_grooming            resting  -0.7922  0.001 -1.0048 -0.5796   True
 antennal_grooming            walking   0.2344 0.0181  0.0219  0.4469   True
 anterior_grooming   foreleg_grooming   0.3951  0.001  0.2378  0.5523   True
 anterior_grooming   hindleg_grooming  -0.2726    0.9  -2.196  1.6509  False
 anterior_grooming posterior_grooming   0.3988  0.001  0.2375  0.5601   True
 anterior_grooming            resting  -0.5693  0.001 -0.6146 -0.5239   True
 anterior_grooming            walking   0.4573  0.001  0.4124  0.5023   True
   foreleg_grooming   hindleg_grooming  -0.6676    0.9 -2.5966  1.2614  False
  foreleg_grooming posterior_grooming   0.0037    0.9 -0.2139  0.2214  False
  foreleg_grooming            resting  -0.9643  0.001 -1.1173 -0.8113   True
  foreleg_grooming            walking   0.0623    0.9 -0.0906  0.2152  False
  hindleg_grooming posterior_grooming   0.6713    0.9  -1.258  2.6006  False
  hindleg_grooming            resting  -0.2967    0.9 -2.2198  1.6264  False
  hindleg_grooming            walking   0.7299    0.9 -1.1932   2.653  False
posterior_grooming            resting   -0.968  0.001 -1.1252 -0.8109   True
posterior_grooming            walking   0.0586    0.9 -0.0985  0.2156  False
           resting            walking   1.0266  0.001  1.0001  1.0531   True
"""
tukey_df[('ab_g', 'ab_p')] #To test behavior
tukey_df

In [ ]:
# All the neurons with significal difference between resting and walking
tukey_df[tukey_df[('r', 'w')]==True].index

In [ ]:
#LEt's plot a matrix with the neurons and their value for behavior
plt.figure(figsize=(16,10))
plt.matshow([resting_means, walking_means, anterior_grooming_means,antennal_grooming_means,foreleg_grooming_means, \
             abdominal_grooming_means,abdominal_pushing_means,posterior_grooming_means,hindleg_grooming_means], fignum=1, aspect='auto')
plt.yticks([0, 1, 2, 3, 4, 5, 6,7,8], states.tolist())
plt.xlabel("neurons number")
plt.show()

In [ ]:
#And now find all neurons that are significant for one behavior to all other behaviors
all_true_neurons = []
for i, neuron in tukey_df.iterrows() : 
    for column in columns :
        cols = [col for col in neuron.index if column in col]
        boolean = all(neuron[cols].values)
        if boolean : all_true_neurons.append({'neuron_'+str(neuron.name) : column})

In [ ]:
# Here we looked for significance with walking behavior 
neurons_walking = []
for i, neuron in tukey_df.iterrows() : 
    for column in columns :
        if column == 'w' :
            cols = [col for col in neuron.index if column in col]
            count_true = 0
            false = ""
            for i in range(len(neuron[cols].values)) :
                if neuron[cols].values[i] : count_true = count_true + 1
                else : false = cols[i][0]
            if count_true >6 : neurons_walking.append({'neuron_'+str(neuron.name) : column+" "+false})
neurons_walking

In [ ]:
def find_behavior(neural_df, behavior_given) :
    # function to plot background behavior activity
    res = []
    for behavior in neural_df['Manual'] :
        if behavior == behavior_given : res.append(400)
        else : res.append(0)
    return res

In [ ]:
#Let's plot neurons that are involved in the resting state
for i in range(12):
    neuron_2 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_2"]-610)/610
    neuron_62 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_62"]-610)/610
    neuron_80 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_80"]-610)/610
    neuron_81 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_81"]-610)/610

    fig, ax = plt.subplots(1,1, figsize=(9.5, 5))
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_2, label="neuron 2")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_62, label="neuron 62")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_80, label="neuron 80")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_81, label="neuron 81")
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'resting'), facecolor='orange', alpha=0.5)
    ax.set_xlabel("t (s)")
    ax.set_ylabel("$\% \Delta F/F$")
    ax.legend()

    ax.spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the right spine/ticks
    ax.spines['right'].set_color('none')
    ax.yaxis.tick_left()

    # set the y-spine
    ax.spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the top spine/ticks
    ax.spines['top'].set_color('none')
    ax.xaxis.tick_bottom()
    plt.title(f'Trial {i}')

In [ ]:
#Let's plot neurons that are involved in the foreleg_grooming state
for i in range(12):
    neuron_25 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_25"]-610)/610
    neuron_36 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_36"]-610)/610

    fig, ax = plt.subplots(1,1, figsize=(9.5, 5))
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_25, label="neuron 25")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_36, label="neuron 36")
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'foreleg_grooming'), facecolor='orange')
    ax.set_xlabel("t (s)")
    ax.set_ylabel("$\% \Delta F/F$")
    ax.legend()

    ax.spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the right spine/ticks
    ax.spines['right'].set_color('none')
    ax.yaxis.tick_left()

    # set the y-spine
    ax.spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the top spine/ticks
    ax.spines['top'].set_color('none')
    ax.xaxis.tick_bottom()
    plt.title(f'Trial {i}')

In [ ]:
#Let's plot neurons that are involved in the abdominal_pushing state
for i in range(12):
    neuron_42 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_42"]-610)/610
    neuron_71 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_71"]-610)/610
    neuron_78 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_78"]-610)/610
    neuron_86 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_86"]-610)/610
    neuron_109 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_109"]-610)/610
    neuron_122 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_122"]-610)/610

    fig, ax = plt.subplots(1,1, figsize=(9.5, 5))
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_42, label="neuron 42")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_71, label="neuron 71")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_78, label="neuron 78")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_86, label="neuron 86")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_109, label="neuron 109")
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_122, label="neuron 122")
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'abdominal_pushing'), facecolor='orange')
    ax.set_xlabel("t (s)")
    ax.set_ylabel("$\% \Delta F/F$")
    ax.legend()

    ax.spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the right spine/ticks
    ax.spines['right'].set_color('none')
    ax.yaxis.tick_left()

    # set the y-spine
    ax.spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the top spine/ticks
    ax.spines['top'].set_color('none')
    ax.xaxis.tick_bottom()
    plt.title(f'Trial {i}')

In [ ]:
#Let's plot neurons that are involved in the antennal_grooming state
for i in range(12):
    neuron_113 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_113"]-610)/610

    fig, ax = plt.subplots(1,1, figsize=(9.5, 5))
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_113, label="neuron 113")
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'antennal_grooming'), facecolor='orange')
    ax.set_xlabel("t (s)")
    ax.set_ylabel("$\% \Delta F/F$")
    ax.legend()

    ax.spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the right spine/ticks
    ax.spines['right'].set_color('none')
    ax.yaxis.tick_left()

    # set the y-spine
    ax.spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the top spine/ticks
    ax.spines['top'].set_color('none')
    ax.xaxis.tick_bottom()
    plt.title(f'Trial {i}')

In [ ]:
#Let's plot neuron 122 with 2 behaviors 
for i in range(12):
    neuron_122 = 100*(neural_data_filtered.loc[(210301, "J1xCI9", 1, i), "neuron_122"]-610)/610

    fig, ax = plt.subplots(1,1, figsize=(9.5, 5))
    ax.plot(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], neuron_122, label="neuron 122")
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'abdominal_pushing'), facecolor='orange')
    ax.fill_between(neural_data_filtered.loc[(210301, "J1xCI9", 1, 0), "t"], 0, find_behavior(neural_df_s[i*4040:i*4040+4040], 'abdominal_grooming'), facecolor='blue')
    ax.set_xlabel("t (s)")
    ax.set_ylabel("$\% \Delta F/F$")
    ax.legend()

    ax.spines['left'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the right spine/ticks
    ax.spines['right'].set_color('none')
    ax.yaxis.tick_left()

    # set the y-spine
    ax.spines['bottom'].set_position(('outward', 3))  # ('axes', -0.02))  # 'zero'

    # turn off the top spine/ticks
    ax.spines['top'].set_color('none')
    ax.xaxis.tick_bottom()
    plt.title(f'Trial {i}')

In [ ]:
# Here we're computing the spearman correlation for each of the neuron with each of the joint to see whether specific
# neuron trigger specific angles.

pos_spearman_corr = []
neg_spearman_corr = []
for neuron in neural_df_s.columns : 
    if neuron == 't' : pass
    else : 
        for angle in behav_data_r.filter(regex="angle") : 
            spearman = spearmanr(neural_df_s[neuron], behav_data_r[angle])
            if (spearman[1]<0.05) & (spearman[0]>0.5) :
                pos_spearman_corr.append([neuron, angle, spearman])
            elif (spearman[1]<0.05) & (spearman[0]<-0.5) :
                neg_spearman_corr.append([neuron, angle, spearman])
print(f'Positive Spearman correlation : {pos_spearman_corr}')
print(f'Negative Spearman correlation :{neg_spearman_corr}')

In [ ]:
# Let's do now a scatter plot between the neurons and angles that have a significant correlation
for pair in pos_spearman_corr :
    plt.scatter(neural_df_s[pair[0]], behav_data_r[pair[1]])
    plt.xlabel(pair[0])
    plt.ylabel(pair[1])
    plt.title('Spearman positive correlation')
    plt.show()
for pair in neg_spearman_corr :
    plt.scatter(neural_df_s[pair[0]], behav_data_r[pair[1]])
    plt.xlabel(pair[0])
    plt.ylabel(pair[1])
    plt.title('Spearman negative correlation')
    plt.show()

### Identifying correlations of low-dimensional signals

In [ ]:
# behaviour_data_reduced : 
# size : (48480*143)

label_dictionary = {'resting':0, 'walking':1, 'anterior_grooming':2, 'antennal_grooming':3, 'foreleg_grooming':4, 'abdominal_grooming':5, 'abdominal_pushing':6, 'posterior_grooming':7, 'hindleg_grooming':8}
labels = behav_data_r["Manual"].map(label_dictionary).to_numpy()
colors = ['#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30', '#4DBEEE', '#A2142F', '#000000', '#1e591e']

labels_reduced = np.copy(labels)
labels_reduced[labels_reduced>2] = 2
colors_reduced = ['#0072BD', '#D95319', '#EDB120']

In [ ]:
# X1 : (n_samples, n_features)
# size : (48480*123)
# 123 components
# 48480 samples

fig, ax = plt.subplots(figsize = (16,12))
scat = ax.scatter(X1[:,0], X1[:,1], s=0.4, c=labels, cmap=mpl.colors.ListedColormap(colors))
ax.set_xlabel("Component 0 [-]")
ax.set_ylabel("Component 1 [-]")
ax.set_title(f"Class-coded scatter-plot of the first two components of the PCA results")

cb = fig.colorbar(scat)
loc = np.arange(0,max(labels),max(labels)/float(len(colors)))
cb.set_ticks(loc)
cb.set_ticklabels(list(label_dictionary.keys()))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (16,12))
scat = ax.scatter(X1[:,0], X1[:,1], s=0.4, c=labels_reduced, cmap=mpl.colors.ListedColormap(colors_reduced))
ax.set_xlabel("Component 0 [-]")
ax.set_ylabel("Component 1 [-]")
ax.set_title(f"Class-coded scatter-plot of the first two components of the PCA results")

cb = fig.colorbar(scat)
loc = np.arange(0,max(labels_reduced),max(labels_reduced)/float(len(colors_reduced)))
cb.set_ticks(loc)
cb.set_ticklabels(['resting', 'walking', 'grooming'])

plt.show()

In [ ]:
# Y1 :
# size : (48480*2)
# 48480 samples
# 2 components

fig, ax = plt.subplots(figsize = (16,12))
scat = ax.scatter(Y1[:,0], Y1[:,1], s=0.4, c=labels, cmap=mpl.colors.ListedColormap(colors))
ax.set_xlabel("Component 0 [-]")
ax.set_ylabel("Component 1 [-]")
ax.set_title(f"Class-coded scatter-plot of the first two components of the PCA results")

cb = fig.colorbar(scat)
loc = np.arange(0,max(labels),max(labels)/float(len(colors)))
cb.set_ticks(loc)
cb.set_ticklabels(list(label_dictionary.keys()))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (16,12))
scat = ax.scatter(Y1[:,0], Y1[:,1], s=0.4, c=labels_reduced, cmap=mpl.colors.ListedColormap(colors_reduced))
ax.set_xlabel("Component 0 [-]")
ax.set_ylabel("Component 1 [-]")
ax.set_title(f"Class-coded scatter-plot of the first two components of the PCA results")

cb = fig.colorbar(scat)
loc = np.arange(0,max(labels_reduced),max(labels_reduced)/float(len(colors_reduced)))
cb.set_ticks(loc)
cb.set_ticklabels(['resting', 'walking', 'grooming'])

plt.show()

In [ ]:
# multi-variate regression model
# regressors: binary behavioural variables (i.e., walking yes/no, grooming yes/no, ...)
# output: neuronal activity (possibly standardised)
MVA_regressors = np.identity(9)[labels]
MVA_output = neural_data.to_numpy()[:,1:]
MVA_score = np.zeros(123)

for i in range(123):
    reg = LinearRegression().fit(MVA_regressors,MVA_output[:,i])
    MVA_score[i] = reg.score(MVA_regressors, MVA_output[:,0])

print(f"Maximum explained variance with MVA : {max(MVA_score):2.2}")

## Classifying behaviour from neuronal activity

In [ ]:
# Data Preparation
nb_trials = 12
neurons_data_manual = neural_df_s.loc[(210301, "J1xCI9", 1), "Manual"].to_numpy()
neurons_data = neural_df_s.loc[(210301, "J1xCI9", 1)].to_numpy()[:,1:-1]
neuron_data_nb = np.zeros((123,48480))
for i in range(123):
    neuron_data_nb[i,:] = neural_df_s.loc[(210301, "J1xCI9", 1), "neuron_"+ str(i)].to_numpy()

# Labeling
walking_yn = np.zeros(48480)
label = np.zeros(48480)

for k in range(48480):
    if neurons_data_manual[k] == "walking":
        walking_yn[k] = 1
        label[k] = 1
    if neurons_data_manual[k] == "abdominal_pushing":
        label[k] = 2
    if neurons_data_manual[k] == "anterior_grooming":
        label[k] = 3
    if neurons_data_manual[k] == "posterior_grooming":
        label[k] = 4
    if neurons_data_manual[k] == "foreleg_grooming":
        label[k] = 5
    if neurons_data_manual[k] == "abdominal_grooming":
        label[k] = 6
    if neurons_data_manual[k] == "hindleg_grooming":
        label[k] = 7
    if neurons_data_manual[k] == "antennal_grooming":
        label[k] = 8
    
# Data for trainning
X_train, X_test, y_train, y_test = train_test_split(neurons_data, label, test_size=0.2)

### Predicting one behaviour

In [ ]:
# Logistic regression on all trials with each neuron at time on walking yes/no
score_all_neuron = np.zeros(123)
for i in range(123):
    logisticRegr = LogisticRegression()
    logisticRegr.fit(neuron_data_nb[i,:].reshape(-1, 1), walking_yn)
    score_all_neuron[i] = logisticRegr.score(neuron_data_nb[i,:].reshape(-1, 1), walking_yn)
print(f"Best neurons : {np.argsort(score_all_neuron)[::-1][:5]}")

In [ ]:
# Score for the 40 first neuron
score_all_neuron[np.argsort(score_all_neuron)[::-1][:40]]

In [ ]:
#Plot for neuron found with logistic regression with only walking
plot_neuron_data(np.argsort(score_all_neuron)[::-1], 2) 

In [ ]:
# logistic regression on all trials with all neuron on walking yes/no
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(neurons_data, walking_yn, test_size=0.2)
logisticRegr = LogisticRegression(max_iter=2000)
logisticRegr.fit(X_train_2, y_train_2)
score = logisticRegr.score(X_test_2, y_test_2)
score = "{:.4f}".format(score)
print(f"Score with it self : {score}")

### Predicting all behaviour

In [ ]:
# Logistic regression  for all neuron all class
logisticRegr = LogisticRegression(max_iter=2000)
logisticRegr.fit(X_train, y_train)
score = logisticRegr.score(X_test, y_test)
score = "{:.4f}".format(score)
print(f"Score with it self : {score}")

In [ ]:
# Coefficient analysis for logistic regression
logisticRegr.coef_.shape
nb_neuron_importance = 10

for i in range(logisticRegr.coef_.shape[0]):
    print(f"For {i}")
    feature = logisticRegr.coef_[i,:]
    sort_feature = np.argsort(feature)[::-1][:nb_neuron_importance]
    print(f"{nb_neuron_importance} important Neuron for the random forest")
    for j in range(nb_neuron_importance):
        print(f"Neuron : {sort_feature[j]}")
    #Plot for neuron found with logistic regression 
    plot_neuron_data(sort_feature, 2)        

In [ ]:
# Random forest classifier
clf = RandomForestClassifier(n_estimators=70)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(f"score : {score}")

In [ ]:
# Feature analysis for random forest
nb_neuron_importance = 10
feature = np.argsort(clf.feature_importances_)[::-1][:nb_neuron_importance]
print(f"{nb_neuron_importance} important Neuron for the random forest")
for i in range(nb_neuron_importance):
    print(f"Neuron : {feature[i]}")

In [ ]:
#Plot for neuron found with random forest
plot_neuron_data(feature, 2)        